
# Task 1: Permutations and Combinations

_Suppose we alter the Lady Tasting Tea experiment to involve twelve cups of tea. Six have the milk in first and the other six having tea in first. A person claims they have the special power of being able to tell whether the tea or the milk went into a cup first upon tasting it. You agree to accept their claim if they can tell which of the six cups in your experiment had the milk in first_.

- _Calculate, using Python, the probability that they select the correct six cups. Here you should assume that they have no special powers in figuring it out, that they are just guessing. Remember to show and justify your workings in code and MarkDown cells._

### Steps:
1. Calculate the total number of ways to select 6 cups from 12.
2. Calculate the successful ways for different scenarios (6/5/4 correct attempts).
3. Sum up the successful ways.
4. Calculate the probability.


In [42]:
import math
from scipy.special import comb 


total_cups = 12  # Total number of cups
milk_cups = 6  # Number of cups with milk added first
tea_cups = 6  # Number of cups with tea added first
all_attempts = comb(total_cups, milk_cups)
probability = 1 / all_attempts

print(f"Probability of selecting all 6 milk-first cups correctly: {probability:.10f}")


Probability of selecting all 6 milk-first cups correctly: 0.0010822511


- _Suppose, now, you are willing to accept one error. Once they select the six cups they think had the milk in first, you will give them the benefit of the doubt should they have selected at least five of the correct cups. Calculate the probability, assuming they have no special powers, that the person makes at most one error._

In [43]:
# Calculate the number of ways she has to select exactly 5 correct and 1 incorrect (combination)
five_correct_cups = comb(6, 5) * comb(6, 1) 

# Calculate the number of ways to select all 6 cups correctly
six_correct_cups = 1  # There is only one way to pick all 6 correctly as only 6 cups have milk in first

# Total successful ways 
successful_ways = five_correct_cups + six_correct_cups

# Probability of selecting at least 5 correct cups
probability_five_cups_correct= successful_ways / all_attempts

print(f"Probability of selecting at least 5 correct cups: {probability_five_cups_correct:.10f}")


Probability of selecting at least 5 correct cups: 0.0400432900


- _Would you accept two errors? Explain_.

Allowing two errors would mean calculating the probability of selecting at least four correct cups out of six, which would increase the probability of a correct guess. However, allowing more errors lowers the test’s reliability, making it easier for the lady to guess the tea that has the tea in first. 

In [44]:
# Calculate the number of ways to select exactly 4 correct and 2 incorrect (combination)
four_correct_cups = comb(milk_cups, 4) * comb(tea_cups, 2) 
successful_ways = four_correct_cups + five_correct_cups + six_correct_cups
probability_four_cups_correct = successful_ways / all_attempts
print(f"Probability of selecting at least 4 correct cups: {probability_four_cups_correct:.10f}")

Probability of selecting at least 4 correct cups: 0.2835497835


## References:
- https://www.geeksforgeeks.org/permutations-and-combinations/
- https://betterexplained.com/articles/easy-permutations-and-combinations/
